<a href="https://colab.research.google.com/github/ericksalignac/clothes-sales-prediction/blob/main/AP3_Gest%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, os, glob, re, glob, io
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
from google.colab import drive
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
from sklearn import *
from matplotlib import style
from sklearn.metrics import mean_absolute_error
from sklearn.tree import *
from sklearn.impute import *
from sklearn.ensemble import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.model_selection import *

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/roupas_de_brechó'

/content/drive/MyDrive/roupas_de_brechó


In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df.head()

,Unnamed: 0,marca,nomeDaPeca,precoComDesconto,precoSemDesconto,Descricao,Tamanho,Condicao,Cores,VENDIDO
0,0,sem marca,\nBlusa Verde Babados\n,64.0,160.0,Blusa verde com modelagem básica. Peça com bab...,m,gentilmente usada,verde,0
1,1,zara,\nSaia Lápis Zara\n,57.0,219.0,Marca: Zara Woman,38,gentilmente usada,"Detalhe: Dois bolsos frontais, forro, zíper tr...",1
2,2,1 a 9,\nJeans Skinny 1 a 9\n,69.0,279.0,"Medidas: cintura 32cm, quadril 43cm, comprime...",38,gentilmente usada,azul,0
3,3,Coach,Sapatilha Monograma,"R$ 250,00","R$ 875,00",Sapatilha caramelo. Possui monograma em toda a...,39,Usado,\nCaramelo \nPreto \n,1
4,4,primeiro item teen,\nShorts Feminino Teen Corações\n,20.97,69.9,"Shorts feminino, com estampa de corações, do P...",m,gentilmente usada,colorido,0


In [ ]:
df_test.head()

,Unnamed: 0,marca,nomeDaPeca,precoComDesconto,precoSemDesconto,Descricao,Tamanho,Condicao,Cores
0,0,Cris Barros,Sandália Nude Holográfica,"R$ 410,00","R$ 1.324,00",Sandália salto alto bloco na cor nude. Possui ...,38,Novo com etiqueta,\nNude \n
1,1,h2n,\nSaia Jeans H2N\n,5.44,79.99,"Saia jeans feminina, cinza. Tecido plano. Mode...",46,gentilmente usada,cinza
2,2,arte & cor,\nVestido Estampado Arte & Cor\n,13.79,109.9,"Vestido feminino, estampado em tons de vermelh...",p,gentilmente usada,Marca: Arte & Cor
3,3,Thelure,Vestido Lilás Trança,"R$ 160,00","R$ 490,00","Vestido mini, na cor lilás. Possui alças finas...",PP,Usado,\nLilás \n
4,4,Triya,Regata Preta Black,"R$ 80,00","R$ 280,00",Regata básica na cor preta com estampa escrita...,M,Usado,\nPreto \n


In [ ]:
df[['precoComDesconto', 'precoSemDesconto']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78331 entries, 0 to 78330
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   precoComDesconto  78331 non-null  object
 1   precoSemDesconto  76821 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [ ]:
# base de dados das labels
y = df['VENDIDO']

In [ ]:
# base de dados das entradas
X = df[['marca','precoComDesconto','precoSemDesconto','Tamanho','Condicao']]

In [ ]:
#  União da base Test e Train para transformações em comum
X = pd.concat([X,df_test[['marca','precoComDesconto','precoSemDesconto','Tamanho','Condicao']]])

In [ ]:
# Remoção de carateres do alfabeto
X['precoComDesconto'] = X['precoComDesconto'].str.extract(r'(\d+[.,\d]*)')
X['precoSemDesconto'] = X['precoSemDesconto'].str.extract(r'(\d+[.,\d]*)')

In [ ]:
# Troca dos caracteres de virgula para ponto nas colunas preço
X['precoComDesconto'] = X['precoComDesconto'].str.replace(',', '.')
X['precoSemDesconto'] = X['precoSemDesconto'].str.replace(',', '.')

In [ ]:
def format_preco(test):
    try:
        test = test.split('.')
        test = test[0] + test[1] + '.' + test[2]
    except:
        test = 0
    return test

In [ ]:
#  Formatando preços com dois pontos  (milhar e centávos)
X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'] = X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'].apply(lambda x: format_preco(x) )
X.loc[X[X['precoSemDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoSemDesconto'] = X.loc[X[X['precoSemDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoSemDesconto'].apply(lambda x: format_preco(x) )

<ipython-input-13-d2b10fd12c11>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'] = X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'].apply(lambda x: format_preco(x) )
<ipython-input-13-d2b10fd12c11>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'] = X.loc[X[X['precoComDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoComDesconto'].apply(lambda x: format_preco(x) )
<ipython-input-13-d2b10fd12c11>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  X.loc[X[X['precoSemDesconto'].str.replace('\d+', '').str.count('.') == 2].index,'precoSemDesconto'] = X.loc[X[X['precoSemDesconto'].str

In [ ]:
#  Atribuindo para valores nulos na coluna 'precoSemDesconto' o valor 0
X.loc[X[X['precoSemDesconto'].isnull()].index,'precoSemDesconto'] = 0

In [ ]:
# Transformando as colunas 'precoSemDesconto' e 'precoComDesconto' no tipo float
X['precoSemDesconto'] = X['precoSemDesconto'].astype(float)
X['precoComDesconto'] = X['precoComDesconto'].astype(float)

In [ ]:
X

,marca,precoComDesconto,precoSemDesconto,Tamanho,Condicao
0,sem marca,64.00,0.0,m,gentilmente usada
1,zara,57.00,219.0,38,gentilmente usada
2,1 a 9,69.00,279.0,38,gentilmente usada
3,Coach,250.00,875.0,39,Usado
4,primeiro item teen,20.97,69.9,m,gentilmente usada
...,...,...,...,...,...
38577,nike,16.70,104.0,p,gentilmente usada
38578,Raia de Goeye,300.00,2992.0,P,Novo com etiqueta
38579,sem etiqueta,11.92,39.9,p,gentilmente usada
38580,asos,60.00,119.0,40,gentilmente usada


In [ ]:
X['Tamanho'].unique()

array(['m', '38', ' 39', 'p', ' 38', ' 36', ' U', 'u', '34', ' M', '36',
       ' 42', 'g', 'pp', '42', ' 35', '40', ' P', '6', nan, '14', ' 34',
       ' 40', ' G', 'gg', '44', '3m', '50', '3', '4', ' PP', '8', '33',
       ' 6M', ' 3M', ' 10M', ' 12M', '12', '37', '35', '48', '16', ' RN',
       '2a', ' GG', ' 37', ' 44', '1', 'rn', '6m', '32', '21', '12m',
       'xgg', '18m', ' 8A', '7', ' 12A', '9m', '46', ' 2A', '5', '2',
       '39', '54', ' 20', ' 1A', ' 10A', ' 18', '10', '29', ' 4A', '30',
       ' 50', ' 6A', ' 48', ' 46', ' 16', '52', '11', ' 17', ' 33', '13',
       '22', ' 14A', '41', '20', '9', ' 21', ' 19', '31', '25', ' 41',
       '58', '17', ' 13', ' 26', '19', '28', '23', '26', ' 14', ' 4',
       '24', '43', ' 3A', ' 32', '56', ' 1', ' XLG', ' 23', ' 15', 'M',
       '27', '18', ' G3', ' 24', '15', ' 54', ' 3', ' 56', ' G4', '45',
       ' 5G', ' 28', '60', ' 27', 'P', ' 52', ' G2', ' 2', ' 4G'],
      dtype=object)

In [ ]:
for i, row in X.iterrows():
    tamanho = row['Tamanho']

    if tamanho in ["1M", "1m"]:
        X['Tamanho'][i] = "1M"
    elif tamanho in ["3M", "3m"]:
        X['Tamanho'][i] = "3M"
    elif tamanho in ["6M", "6m"]:
        X['Tamanho'][i] = "6M"
    elif tamanho in ["9M", "9m"]:
        X['Tamanho'][i] = "9M"
    elif tamanho in ["10M", "10m"]:
        X['Tamanho'][i] = "10M"
    elif tamanho in ["12M", "12m"]:
        X['Tamanho'][i] = "12M"
    elif tamanho in ["18M", "18m"]:
        X['Tamanho'][i] = "18M"
    elif tamanho in ["1A", "1a", "1"]:
        X['Tamanho'][i] = "1"
    elif tamanho in ["2A", "2a", "2"]:
        X['Tamanho'][i] = "2"
    elif tamanho in ["3A", "3a"]:
        X['Tamanho'][i] = "3"
    elif tamanho in ["4A", "4a", "4"]:
        X['Tamanho'][i] = "4"
    elif tamanho in ["6A", "6a", "6"]:
        X['Tamanho'][i] = "6"
    elif tamanho in ["8A", "8a", "8"]:
        X['Tamanho'][i] = "8"
    elif tamanho in ["10A", "10a", "10"]:
        X['Tamanho'][i] = "10"
    elif tamanho in ["12A", "12a", "12"]:
        X['Tamanho'][i] = "12"
    elif tamanho in ["14A", "14a", "14"]:
        X['Tamanho'][i] = "14"
    elif tamanho in ["Rn", "RN", "rn", "RN"]:
        X['Tamanho'][i] = "RN"
    elif tamanho in ["PP", "pp", "34"]:
        X['Tamanho'][i] = "34"
    elif tamanho in ["P", "p", "36"]:
        X['Tamanho'][i] = "36"
    elif tamanho in ["M", "m", "38"]:
        X['Tamanho'][i] = "38"
    elif tamanho in ["G", "g", "40"]:
        X['Tamanho'][i] = "40"
    elif tamanho in ["G2", "GG", "Gg", "gg", "42"]:
        X['Tamanho'][i] = "42"
    elif tamanho in ["GGG", "xgg", "XLG", "44"]:
        X['Tamanho'][i] = "44"
    elif tamanho in ["GGGG", "G4", "46"]:
        X['Tamanho'][i] = "46"
    elif tamanho in ["U", "u", "U"]:
        X['Tamanho'][i] = "U"

print(X)

<ipython-input-18-032890002976>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Tamanho'][i] = "38"
<ipython-input-18-032890002976>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Tamanho'][i] = "36"
<ipython-input-18-032890002976>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Tamanho'][i] = "U"
<ipython-input-18-032890002976>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

                    marca  precoComDesconto  precoSemDesconto Tamanho  \
0               sem marca             64.00               0.0      38   
1                    zara             57.00             219.0      46   
2                   1 a 9             69.00             279.0      36   
3                   Coach            250.00             875.0      39   
4      primeiro item teen             20.97              69.9      38   
...                   ...               ...               ...     ...   
38577                nike             16.70             104.0      36   
38578       Raia de Goeye            300.00            2992.0       P   
38579        sem etiqueta             11.92              39.9      36   
38580                asos             60.00             119.0      40   
38581           sem marca             25.00              49.9      38   

                Condicao  
0      gentilmente usada  
1      gentilmente usada  
2      gentilmente usada  
3              

In [ ]:
X['Tamanho'].unique()

array(['38', '46', '36', ' 39', ' 38', ' 36', 'U', '34', '10', ' M', '40',
       ' 42', '44', ' 35', ' P', '6', '42', '14', ' G', ' U', '3', '4',
       ' 40', ' 34', ' PP', ' 12M', '2', '18M', '3M', '35', '12', '48',
       '37', '16', ' RN', ' 3M', ' 37', ' 44', ' GG', 'RN', '8', '12M',
       nan, '21', ' 6M', ' 8A', '7', ' 2A', '1', '5', '6M', '39', '54',
       ' 18', '9M', ' 6A', ' 48', '33', ' 46', ' 10M', ' 10A', ' 33',
       ' 16', ' 1A', '20', '9', '52', ' 19', '31', '11', '25', '41',
       ' 4A', ' 17', '50', ' 26', '19', '26', ' 14', ' 4', ' 12A', ' 50',
       '23', '43', '17', '58', ' 21', ' 3A', ' 32', '56', ' 20', ' 1',
       '28', '22', '13', ' 15', ' 23', '24', '18', '27', ' 14A', '15',
       ' 54', ' 41', ' 3', ' 56', '29', '32', '30', ' G4', '45', ' 5G',
       ' 28', ' G3', '60', ' 27', ' XLG', ' G2', ' 2', ' 4G'],
      dtype=object)

In [ ]:
X.replace('\n','',regex=True, inplace=True)

In [ ]:
X['Condicao'] = X['Condicao'].str.replace('Novo com etiqueta', 'novo com etiqueta', case=False)
X['Condicao'] = X['Condicao'].str.lower()

In [ ]:
X['marca'] = X['marca'].str.lower()

In [ ]:
# enc = OneHotEncoder(handle_unknown='ignore')
# transformed = enc.fit_transform(X)
# transformed

In [ ]:
X = pd.get_dummies(X, columns=['Condicao'])
X

,marca,precoComDesconto,precoSemDesconto,Tamanho,Condicao_gentilmente usada,Condicao_nova com etiqueta,Condicao_novo com etiqueta,Condicao_novo sem etiqueta,Condicao_usado
0,sem marca,64.00,0.0,38,1,0,0,0,0
1,zara,57.00,219.0,46,1,0,0,0,0
2,1 a 9,69.00,279.0,36,1,0,0,0,0
3,coach,250.00,875.0,39,0,0,0,0,1
4,primeiro item teen,20.97,69.9,38,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
38577,nike,16.70,104.0,36,1,0,0,0,0
38578,raia de goeye,300.00,2992.0,P,0,0,1,0,0
38579,sem etiqueta,11.92,39.9,36,1,0,0,0,0
38580,asos,60.00,119.0,40,1,0,0,0,0


In [ ]:
le = LabelEncoder()

X['marca'] = le.fit_transform(X.marca.values)
X['Tamanho'] = le.fit_transform(X.Tamanho.values)
# #X['nomeDaPeca'] = le.fit_transform(X.nomeDaPeca)
# X['Condicao'] = le.fit_transform(X.Condicao.values)

In [ ]:
# Separando os datasets de treino
X_treino = X.iloc[:78331]

In [ ]:
# Separador de base original para base de treinamento (train) e validação (test)
X_train, X_test, y_train, y_test = train_test_split(X_treino, y, test_size=0.33, random_state=42)

In [ ]:
# Realizando a busca de hiperparâmetros para a árvore de decisão
tuned_parameters = [{'max_depth': [1,2,3,4,5,6,7,8,10,12,14],
                     'min_samples_split': [2,3,4,5,6,7,8,10,12,14],
                     'criterion' : ['gini','entropy',]
                     }]
clf = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), tuned_parameters)
clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   param_distributions=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8,
                                                       10, 12, 14],
                                         'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                               8, 10, 12,
                                                               14]}])

In [ ]:
# Imprimindo os melhores parâmetros encontrados
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

# Calculando métricas de desempenho para o modelo de árvore de decisão com os melhores parâmetros
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)
print()

Best parameters set found on development set:

{'min_samples_split': 14, 'max_depth': 7}

Grid scores on development set:

0.628 (+/-0.003) for {'min_samples_split': 14, 'max_depth': 7}
0.624 (+/-0.004) for {'min_samples_split': 6, 'max_depth': 3}
0.626 (+/-0.005) for {'min_samples_split': 8, 'max_depth': 5}
0.626 (+/-0.003) for {'min_samples_split': 8, 'max_depth': 6}
0.611 (+/-0.009) for {'min_samples_split': 5, 'max_depth': 1}
0.628 (+/-0.003) for {'min_samples_split': 10, 'max_depth': 7}
0.626 (+/-0.003) for {'min_samples_split': 14, 'max_depth': 6}
0.625 (+/-0.008) for {'min_samples_split': 6, 'max_depth': 8}
0.628 (+/-0.003) for {'min_samples_split': 6, 'max_depth': 7}
0.627 (+/-0.010) for {'min_samples_split': 14, 'max_depth': 10}
MSE: 0.3706



In [ ]:
print('Score no conjunto de treino: {:.1%}'.format(clf.score(X_train, y_train)))
print('Score no conjunto de teste: {:.1%}'.format(clf.score(X_test, y_test)))

Score no conjunto de treino: 63.7%
Score no conjunto de teste: 62.9%
